Data downloaded from ENTSOE API:
https://github.com/EnergieID/entsoe-py

In [23]:
from entsoe import EntsoePandasClient
import pandas as pd
import numpy as np

In [19]:
client = EntsoePandasClient(api_key='a6160036-4d49-4c39-960f-99c3c690b6da', retry_count=60, retry_delay=10)

start = pd.Timestamp('20220601', tz='Europe/Copenhagen')
end = pd.Timestamp('20220630', tz='Europe/Copenhagen')
country_codes = ['DK_1', 'DK_2', 'SE_1', 'SE_2', 'SE_3', 'SE_4', 'NO_1', 'NO_2', 'NO_3', 'NO_4', 'NO_5', 'FI']
# Check and confirm these, in order to get the correct data for connected data points
country_connections = {'DK_1': ['DK_2', 'NO_1'], 'DK_2': ['DK_1', 'SE_4'], 'SE_1': ['SE_2', 'NO_1'], 'SE_2': ['SE', 'NO'], 'SE_3': ['SE', 'FI'], 'SE_4': ['SE', 'PL'], 'NO_1': ['NO', 'SE'], 'NO_2': ['NO', 'SE'], 'NO_3': ['NO', 'FI'], 'NO_4': ['NO', 'RU'], 'NO_5': ['NO', 'NL'], 'FI': ['FI', 'SE']}

In [65]:
# empty dataframes
day_ahead_prices = pd.DataFrame()
crossborder_flows = pd.DataFrame()
scheduled_exchanges = pd.DataFrame()
net_transfer_capacity = pd.DataFrame()
aggregate_water_reservoirs_hydro_storage = pd.DataFrame()
load = pd.DataFrame()
load_forecast = pd.DataFrame()
load_and_forecast = pd.DataFrame()
generation_forecast = pd.DataFrame()
wind_and_solar_forecast = pd.DataFrame()

In [66]:
for country_code in country_codes:
    # day ahead prices
    day_ahead = pd.DataFrame(client.query_day_ahead_prices(country_code, start=start, end=end))
    day_ahead['country_code'] = country_code
    day_ahead_prices = pd.concat([day_ahead_prices, day_ahead])
    # load 
    ld = pd.DataFrame(client.query_load(country_code, start=start, end=end))
    ld['country_code'] = country_code
    load = pd.concat([load, ld])
    # load forecast
    ld_fc = pd.DataFrame(client.query_load_forecast(country_code, start=start, end=end))
    ld_fc['country_code'] = country_code
    load_forecast = pd.concat([load_forecast, ld_fc])
    # load and forecast
    ld_fc = pd.DataFrame(client.query_load_and_forecast(country_code, start=start, end=end))
    ld_fc['country_code'] = country_code
    load_and_forecast = pd.concat([load_and_forecast, ld_fc])
    # generation forecast
    gen_fc = pd.DataFrame(client.query_generation_forecast(country_code, start=start, end=end))
    gen_fc['country_code'] = country_code
    generation_forecast = pd.concat([generation_forecast, gen_fc])
    # wind and solar forecast
    ws_fc = pd.DataFrame(client.query_wind_and_solar_forecast(country_code, start=start, end=end, psr_type=None))
    ws_fc['country_code'] = country_code
    wind_and_solar_forecast = pd.concat([wind_and_solar_forecast, ws_fc])


    #crossborder_flows = pd.concat([crossborder_flows, client.query_crossborder_flows(country_code_from, country_code_to, start=start, end=end)]) # missing country_code_to
    #scheduled_exchanges = pd.concat([scheduled_exchanges, client.query_scheduled_exchanges(country_code_from, country_code_to, start=start, end=end)]) # missing country_code_to
    #net_transfer_capacity = pd.concat([net_transfer_capacity, client.query_net_transfer_capacity_dayahead(country_code_from, country_code_to, start=start, end=end)]) # missing country_code_to
    #aggregate_water_reservoirs_hydro_storage = pd.concat([aggregate_water_reservoirs_hydro_storage, client.query_aggregate_water_reservoirs_and_hydro_storage(country_code, start=start, end=end)]) # not matching data
    

In [67]:
data_frame_lists = ['day_ahead_prices', 'load', 'load_forecast', 'load_and_forecast', 'generation_forecast', 'wind_and_solar_forecast'] #  'crossborder_flows', 'scheduled_exchanges', 'net_transfer_capacity', 'aggregate_water_reservoirs_hydro_storage',
# save dataframes to csv
for data_frame_list in data_frame_lists:
    eval(data_frame_list).to_csv(f'data/{data_frame_list}.csv')